# Setup: Connectivity

In [ ]:

from katana import remote
   #
from IPython.display import display
from ipywidgets import Layout, Textarea, VBox
   #
from libraries.SupportFor34 import f_init, f_enrich
    
    
my_client, my_graph = f_init()


print("My Client: %s" % (my_client))
print("My Graph:  %s" % (my_graph ))
   #
print()


#  Step 01: Enter a new Medical Description

In [1]:

#  Create a text entry area
#

l_layout   = Layout(flex = "0 1 auto", height = "100px", min_height = "100px", width = "auto")
l_text01   = Textarea(value="Odd mole on my ear", layout = l_layout, description = "Enter Text: ")
   #
l_vbox     = VBox([l_text01], layout = Layout(flex = "1 1 auto", width = "auto"))
display(l_text01)


NameError: name 'Layout' is not defined

In [ ]:

#  Enrich/decode text entry from above
#

l_return, l_pvid = f_enrich(l_text01.value.lower(), my_graph)

print("Unique Patient Visit Id: %s" % (l_pvid))
print("")
print("Medical Description encoding:")
print("")
display(l_return)


# Step 02:  Examine the Graph

In [ ]:

#  Produce a visual graph

from katana_visualization_widget import GraphVisOptions, NodeVisOption, EdgeVisOption, ANY

l_options = GraphVisOptions(
   node_options = [
      NodeVisOption("PatientVisit",     label="short_text"       ),
      NodeVisOption("UmlsEntity",       label="id"               ),
      NodeVisOption("UmlsVocabulary",   label="id"               ),
   ],
   #  edge_options = [
   #     EdgeVisOption(["VISIT_CONTAINS"], label="start_id"),
   #     EdgeVisOption(["ALSO_CODED_AS" ], label="start_id"),
   #  ]
   )
    
    
l_query  = """

   MATCH (pv: PatientVisit) - [vc: VISIT_CONTAINS] -> (ue: UmlsEntity) - [aca: ALSO_CODED_AS] -> (uv: UmlsVocabulary)
   WHERE
      (pv.id = '{0}')
   RETURN pv, vc, ue, aca, uv
   
   """.format(l_pvid)

l_result = my_graph.query_unpaginated(l_query, contextualize=True)
    
l_result.view(graph_vis_options = l_options)


<div> 
<img src="./01_Images/Results_03.png" alt="Drawing" style="width: 1600px;"/>
</div>

In [ ]:
# #########################################################################

In [ ]:

#  Create a text entry area
#

l_layout   = Layout(flex = "0 1 auto", height = "100px", min_height = "100px", width = "auto")
l_text02   = Textarea(value="UMLS/C0027962", layout = l_layout, description = "Enter Text: ")
   #
l_vbox     = VBox([l_text01], layout = Layout(flex = "1 1 auto", width = "auto"))
display(l_text02)



In [ ]:

#  Produce a visual graph

from katana_visualization_widget import GraphVisOptions, NodeVisOption, EdgeVisOption, ANY

l_options = GraphVisOptions(
   node_options = [
      NodeVisOption("PatientVisit",     label="id"),
      NodeVisOption("UmlsEntity",       label="id"),
      NodeVisOption("UmlsVocabulary",   label="id"),
   ],
   #  edge_options = [
   #     EdgeVisOption(["VISIT_CONTAINS"], label="start_id"),
   #     EdgeVisOption(["ALSO_CODED_AS" ], label="start_id"),
   #  ]
   )

l_query  = """

   MATCH (pv: PatientVisit) - [vc: VISIT_CONTAINS] -> (ue: UmlsEntity) - [aca: ALSO_CODED_AS] -> (uv: UmlsVocabulary)
   WHERE
      (ue.id = '{0}')
   RETURN pv, vc, ue, aca, uv
   LIMIT 100
   
   """.format(l_text02.value)

l_result = my_graph.query_unpaginated(l_query, contextualize=True)
    
l_result.view(graph_vis_options = l_options)


<div> 
<img src="./01_Images/Results_04.png" alt="Drawing" style="width: 1600px;"/>
</div>

In [ ]:
# #########################################################################

In [ ]:

#  Checking just nodes
#

l_query  = """

   MATCH (n)
   WHERE NOT n.id STARTS WITH 'XX-'                   //  A value we use as a placeholder
   RETURN n
   LIMIT 100
   
   """.format()

l_result = my_graph.query_unpaginated(l_query, contextualize=False)
    
print(l_result)



In [ ]:

#  Checking just edges
#

l_query  = """

   MATCH (n) - [r] -> (m)
   RETURN r
   LIMIT 100
   
   """.format()

l_result = my_graph.query_unpaginated(l_query, contextualize=False)
    
print(l_result)


In [ ]:
# #########################################################################

In [ ]:

#  Metadata on our Nodes,
#
#  Sample output, smaller load,
#
#     +----+------------------+-------+
#     |    | label            |   cnt |
#     |----+------------------+-------|
#     |  0 | Keyword          | 10443 |
#     |  1 | MedicalSpecialty |    40 |
#     |  2 | PatientVisit     |  4999 |
#     |  3 | UmlsEntity       | 20073 |
#     |  4 | UmlsVocabulary   | 40148 |
#     +----+------------------+-------+

#  Sample output, larger load,
#
#     +----+------------------+---------+
#     |    | label            |     cnt |
#     |----+------------------+---------|
#     |  0 | Keyword          |   10443 |
#     |  1 | MedicalSpecialty |      40 |
#     |  2 | PatientVisit     |    4999 |
#     |  3 | UmlsEntity       | 4661972 |
#     |  4 | UmlsVocabulary   | 3365900 |
#     +----+------------------+---------+


In [ ]:

#  Checking just nodes
#

l_query  = """

   MATCH (n: UmlsEntity)
   RETURN n
   LIMIT 20
   
   """.format()

l_result = my_graph.query_unpaginated(l_query, contextualize=False)
    
print(l_result)

